In [2]:
from ultralytics import YOLO
import cv2
import pytesseract

In [1]:
from ultralytics import YOLO

model = YOLO(model="yolov8m.pt")

results = model.train(data="datasets/data.yaml", epochs=100, imgsz=640, batch=4, workers=0)

New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.213 🚀 Python-3.13.7 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 3769MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms

KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO

model = YOLO("runs/detect/train2/weights/last.pt")

results = model.train(resume=True)

New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.213 🚀 Python-3.13.7 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 3769MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/train6/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, 

RuntimeError: Checkpoint /home/dlanb/Documents/MyProgects/Recognition number plates/runs/detect/train6/weights/last.pt is corrupted with NaN/Inf weights

In [ ]:
# OCR Function
def perform_ocr(image, detections):
    """
    Perform OCR on cropped regions from the detected bounding boxes and include class names.
    """
    for i, detection in enumerate(detections):
        # Extract bounding box and class name
        x1, y1, x2, y2, class_name = detection
        cropped_image = image[int(y1):int(y2), int(x1):int(x2)]
        
        # Preprocess for better OCR results
        gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
        
        # Perform OCR using Tesseract
        text = pytesseract.image_to_string(binary, lang='eng')
        print(f"Class '{class_name}' detected: {text.strip()}")

        # Save or display cropped regions (optional)
    #     cv2.imshow(f"Region - {class_name}", binary)
    #     cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [ ]:
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

In [ ]:
model_vehicle = YOLO('models/')
model_lisence_plates = YOLO('models/yolo_lisence_plates.pt')

import os
HOME = os.getcwd()  # Getting the current working directory

# Path to the images folder
category = "images"
images_folder = f"{HOME}/{category}"
target_classes = [1]


# image = cv2.imread('nomerogram_data/image4.png')
# Run inference
# results = model(image)[0]  # Get the first result (for a single image)

# Draw results on the image
# annotated_image = results.plot()  # This uses Ultralytics' built-in plotting

# Display the image
#cv2.imshow("YOLO Custom Detection", annotated_image)
#cv2.waitKey(0)  # Wait for a key press to close
#cv2.destroyAllWindows()

# Optionally, save it
#cv2.imwrite('my_custom_result.jpg', annotated_image)

# Iterate through all files in the images folder
for file_name in os.listdir(images_folder):
    if file_name.endswith(('.jpg', '.jpeg', '.png')):  # Check for image files
        file_path = os.path.join(images_folder, file_name)

        # Load the image
        image = cv2.imread(file_path)

        # Run the YOLO model on the current image
        results_vehicles = model_vehicle(file_path)[0]
        # Extract bounding box coordinates and class names
        detections = []
        for box in results_vehicles.boxes.data.tolist():
            x1, y1, x2, y2, conf, cls = box[:6]
            class_name = results_vehicles.names[int(cls)]  # Map class index to class name
            print(class_name)

            vehicle_crop_img = image[int(y1):int(y2), int(x1):int(x2)] 
            results_lisence = model_lisence_plates(vehicle_crop_img)

            for box in results_lisence.boxes.data.tolist():
                x1, y1, x2, y2, conf, cls = box[:6]
                print(class_name)
                class_name = results_lisence.names[int(cls)]
                if class_name == "license plate":
                    lisence_crop_img = image[int(y1):int(y2), int(x1):int(x2)] 
                    cv2.imshow('cropped', lisence_crop_img)
                    text = pytesseract.image_to_string(lisence_crop_img)
                    print(text)

        
        print(f"Detected objects for {file_name}: {detections}")
        # Perform OCR on detected regions
        # perform_ocr(image, detections)

print("Processing complete for all images!")   


image 1/1 /home/dlanb/Documents/MyProgects/Recognition number plates/images/image0.png: 480x640 2 cars, 1 license plate, 20.4ms
Speed: 2.4ms preprocess, 20.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


error: OpenCV(4.12.0) /io/opencv/modules/highgui/src/window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
